In [ ]:
import torch
from source.utils.config_manager import ConfigManager
from source.utils.label_mapper import LabelMapper
from source.utils import save_to_kaggle

# transformers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loaders
from source.dataloaders.cnn_data_loaders_factory import CNNDataLoaderFactory


# networks
from source.pre_trained.efficient_net import PretrainedEff_cnn



In [ ]:
dataset_name = 'cifar10'
dataset_path = ConfigManager.get_dataset_path(dataset_name)
loader = CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_transformer_kaggle(), TestTransformersFactory.get_transformer_efficient())
test_loader = loader.get_test_loader(64)

In [ ]:
model1 = PretrainedEff_cnn()
model1.load_state_dict(torch.load("C:\\Users\\gosia\\Desktop\\studia\\mgr_sem_1\\DL\\cifar10-computer_vision\\source\\pretrained_results\\efficient.pth",map_location=torch.device('cpu')))
model1.eval()

In [ ]:
device = 'cuda'
outputs = model1.predict(test_loader, device,LabelMapper.get_cifar10_mapper()) 

In [ ]:
save_to_kaggle(300000,outputs, "efficient")